# SLTimer Demo

In this notebook we will show how `SLTimer` can be used to estimate a time delay from some example data.

## 1. Obtaining PyCS and its Sample Data 

We will use the "demo1" 4-image light curve data that comes with the `PyCS` repository. Let's download this and use `PyCS` to analyze it, before showing the same operations performed by SLTimer.

This function `imports` the operating system and the url library package which enables us to collect URLs.

In [ ]:
import os, urllib
from __future__ import print_function

This script defines our variables for `webdir` and `rdbfile`. `webdir` makes a copy of PyCS from GitHub so we can utilize their data. `rdbfile` calls trialcurves.txt, the text file that holds the data that we are going to process. `url` concatenates the paths of `webdir` and `rdbfile`. If rdbfile is not there, use url to copy the file locally.

In [ ]:
webdir = 'https://raw.githubusercontent.com/COSMOGRAIL/PyCS/master/demo/demo1/data/'
rdbfile = 'trialcurves.txt'
    
url = os.path.join(webdir, rdbfile)
if not os.path.isfile(rdbfile):
    urllib.urlretrieve(url, rdbfile)

This script will count the number of lines in the rdbfile. 

In [ ]:
!wc -l $rdbfile

In this first script we import `PyCS`, the software needed to process the data. We also call `matplotlib` to be used later as it is needed.

In [2]:
import pycs
%matplotlib inline

## 2. Displaying the Light Curve Data

This script calls the data from the rdbfile, in this case from a simple text file with headers. (Note: other formats are supported as well.)

In [ ]:
lcs = [
        pycs.gen.lc.rdbimport(rdbfile, 'A', 'mag_A', 'magerr_A', "Trial"),
        pycs.gen.lc.rdbimport(rdbfile, 'B', 'mag_B', 'magerr_B', "Trial"),
        pycs.gen.lc.rdbimport(rdbfile, 'C', 'mag_C', 'magerr_C', "Trial"),
        pycs.gen.lc.rdbimport(rdbfile, 'D', 'mag_D', 'magerr_D', "Trial")
]

Let's add some color to this plot! This script gives each curve a different color. 

In [ ]:
pycs.gen.mrg.colourise(lcs) 

This script shifts the data by the "true" time shifts, for display purposes. We will find time shifts for ourselves later in the programming.

In [ ]:
lcs[1].shifttime(-5.0)
lcs[2].shifttime(-20.0)
lcs[3].shifttime(-70.0)

Now to display our plot! 

In [ ]:
pycs.gen.lc.display(lcs)

The most *IMPORTANT* step in this process. Write this information to a pickle file using the code below.

In [ ]:
pycs.gen.util.writepickle(lcs, "trialcurves.pkl")

In further scripts, you can now import the data by reading this file. 

In [3]:
lcs = pycs.gen.util.readpickle("trialcurves.pkl")

Read trialcurves.pkl


We will now undo these shifts, and from now on we will "forget" about the true delays. 

In [4]:
for l in lcs:
        l.resetshifts()

You can do a variety of things with this file to find out more information. For example, running the script below will provide you with the number of points, gap length, shifts, median, mean, maximum, minimum, and the colour it is plotted in.  

In [5]:
for l in lcs: print(l.longinfo())

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
	[Trial/A]
192 points (total), 0 of which are masked
4 seasons (gap: >60), gap length : 164.0 +/- 33.8 days
Sampling : median 4.0, mean 4.4, max 25.1, min 0.83 days
Shifts : (0.00000,0.00000,0.00) [days, mag, flux]
Colour : red
Common properties : 
   All properties : 
Comments :
   Imported from trialcurves.txt, columns (1, 2, 3)
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
	[Trial/B]
192 points (total), 0 of which are masked
4 seasons (gap: >60), gap length : 164.0 +/- 33.8 days
Sampling : median 4.0, mean 4.4, max 25.1, min 0.83 days
Shifts : (0.00000,0.00000,0.00) [days, mag, flux]
Colour : blue
Common properties : 
   All properties : 
Comments :
   Imported from trialcurves.txt, columns (1, 4, 5)
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
	[Trial/C]
192 points (total

To save this figure to a file: 

In [ ]:
pycs.gen.lc.display(lcs, filename="fig_trialcurves.pdf")

Another thing we can do is export the data into a text file called "out_trialcurves.txt". In this case, since we have not altered the original data, this file will contain the same information as trialcurves.txt.

In [ ]:
for l in lcs:
    l.resetshifts() 
    
pycs.gen.util.multilcsexport(lcs, "out_trialcurves.txt", separator="\t", verbose=True, properties=None)

## 3. Factoring Microlensing into the Light Curves and Creating the Free-Knot Spline Fit

The script optimizes the spline. It is a simple attempt to get a multi-purpose free-knot spline method. It is within this code that all of the fine-tuning methods happen. This is done by defining the curve shifting functions that will be used in the scripts. This script optimizes the spline three times; twice as rough with two different knotsteps and once as fine. 

In [ ]:
def spl(lcs):
    spline = pycs.spl.topopt.opt_rough(lcs, nit=5, knotstep=50)
    for l in lcs:
        l.resetml()
    spline = pycs.spl.topopt.opt_rough(lcs, nit=5, knotstep=30)
    spline = pycs.spl.topopt.opt_fine(lcs, nit=10, knotstep=20)
    return spline

This line will print the estimated shifts from before, used to align the different light curves to make them easier to read in our plot. 

In [ ]:
for l in lcs: print (l)

Let's try  curve shifting without correcting for variability, just to see what it looks like. This will be done by using the free-knot spline technique.

In [ ]:
spline = spl(lcs)

This script will display our new graph. Remember, it won't look very good, as the curves do not overlap without a microlensing model. 

In [ ]:
pycs.gen.lc.display(lcs, [spline])

Now let's print "Time Delays:" to make our work a bit easier to read. We will then compute the current time delays as from the current time shifts of each curve. 

In [ ]:
print("Time Delays:")
print (pycs.gen.lc.getnicetimedelays(lcs, separator="\n", sorted=True))

To get even better results, let's factor in the microlensing models (e.g., polynomials) to our light curves. This choice is just an illustration. 

In [ ]:
pycs.gen.polyml.addtolc(lcs[1], nparams=2, autoseasonsgap=60.0)
pycs.gen.polyml.addtolc(lcs[2], nparams=3, autoseasonsgap=600.0)
pycs.gen.polyml.addtolc(lcs[3], nparams=3, autoseasonsgap=600.0)

Now, let's try the free-knot spline optimization again. The result will be much better. 

In [ ]:
spline = spl(lcs) 

Let's display our graph!

In [ ]:
pycs.gen.lc.display(lcs, [spline])

Let's print the new time delays, so we can see the difference once microlensing is factored in. Compare our previous time delay output to this output. 

In [ ]:
print("Time Delays:")
print (pycs.gen.lc.getnicetimedelays(lcs, separator="\n", sorted=True))

This script factors in the microlensing.

In [ ]:
pycs.gen.splml.addtolc(lcs[0], knotstep=150)
pycs.gen.splml.addtolc(lcs[1], knotstep=150)
pycs.gen.splml.addtolc(lcs[2], knotstep=150)
pycs.gen.splml.addtolc(lcs[3], knotstep=150)

This script redefines `spline`. 

In [ ]:
spline = spl(lcs)

This script displays our work in a PDF file for consistent readability, adding specifications to the plot to make it easy to understand. 

In [ ]:
pycs.gen.lc.display(lcs, [spline], knotsize=0.01, figsize=(20, 7), jdrange=(53900, 55500),filename="fig_modelfit.pdf")

All of this work will now be written to a pickle file which will be used later. This file includes the lcs and their optimized microlensing splines. 

In [ ]:
pycs.gen.util.writepickle((lcs, spline), "optspline.pkl")

## 4. Creating Copies of the Data to find the Time Delay Uncertanties

This script makes 200 copies of the data. These will be used to evaluate the instrinic variance of the optimizer and compute the best point estimates for the delays. 

In [6]:
pycs.sim.draw.multidraw(lcs, onlycopy=True, n=5, npkl=4, simset="copies")

Now thowing dice into sims_copies ...
The directory exists, I'll add my new curves.
Input shifts :
A    +0.00 | B    +0.00 | C    +0.00 | D    +0.00
Input delays :
AB   +0.00 | AC   +0.00 | AD   +0.00 | BC   +0.00 | BD   +0.00 | CD   +0.00
Preparing 5 identical copies for pkl 1/4 ...
Wrote sims_copies/1_1466116662.92844.pkl
Preparing 5 identical copies for pkl 2/4 ...
Wrote sims_copies/2_1466116662.95432.pkl
Preparing 5 identical copies for pkl 3/4 ...
Wrote sims_copies/3_1466116662.97396.pkl
Preparing 5 identical copies for pkl 4/4 ...
Wrote sims_copies/4_1466116663.00248.pkl


This makes 1000 synthetic light curves with known true time delays, starting from the generative model. For this step we don't need the "raw" data. Instead, we wil use the the "optimized" light curves, with their shifts and microlensing models from prior steps, and the spline representing the intrinsic QSO variability. 

In [7]:
(modellcs, modelspline)  = pycs.gen.util.readpickle("optspline.pkl")

Read optspline.pkl


Before running our simulation, let's define our tweaks, the small scale extrinsic variability, used to generated the synthetic curves. Something to be noted for the future is that these beta, sigma, and fmin are *assumptions*, these values can be changed. 

In [8]:
def Atweakml(lcs):
    return pycs.sim.twk.tweakml(lcs, beta=-1.5, sigma=0.25, fmin=1/500.0, fmax=None, psplot=False)

def Btweakml(lcs):
    return pycs.sim.twk.tweakml(lcs, beta=-1.0, sigma=0.9, fmin=1/500.0, fmax=None, psplot=False)

def Ctweakml(lcs):
    return pycs.sim.twk.tweakml(lcs, beta=-1.0, sigma=1.5, fmin=1/500.0, fmax=None, psplot=False)

def Dtweakml(lcs):
    return pycs.sim.twk.tweakml(lcs, beta=-0.0, sigma=4.5, fmin=1/500.0, fmax=None, psplot=False)

At this place, *only* if you know what you are doing, you can manually adjust the microlensing or the delays. If not, run this script. This will run 20 simulations for each element in the pickle file. 

In [9]:
pycs.sim.draw.saveresiduals(modellcs, modelspline)
pycs.sim.draw.multidraw(modellcs, modelspline, n=10, npkl=10, simset="1Kset1",
        truetsr=8.0, tweakml=[Atweakml, Btweakml, Ctweakml, Dtweakml])

Now thowing dice into sims_1Kset1 ...
The directory exists, I'll add my new curves.
Input shifts :
A    -7.64 | B   -12.44 | C   -28.36 | D   -76.26
Input delays :
AB   -4.80 | AC  -20.72 | AD  -68.62 | BC  -15.92 | BD  -63.82 | CD  -47.90
Drawing 10 simulations for pkl 1/10 ...


/Users/milanwilliams/PyCS/pycs/gen/spl.py:342: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (self.t == None):


Wrote sims_1Kset1/1_1466116675.50352.pkl
Drawing 10 simulations for pkl 2/10 ...
Wrote sims_1Kset1/2_1466116676.18533.pkl
Drawing 10 simulations for pkl 3/10 ...
Wrote sims_1Kset1/3_1466116677.01582.pkl
Drawing 10 simulations for pkl 4/10 ...
Wrote sims_1Kset1/4_1466116677.75231.pkl
Drawing 10 simulations for pkl 5/10 ...
Wrote sims_1Kset1/5_1466116678.35378.pkl
Drawing 10 simulations for pkl 6/10 ...
Wrote sims_1Kset1/6_1466116678.94336.pkl
Drawing 10 simulations for pkl 7/10 ...
Wrote sims_1Kset1/7_1466116679.54376.pkl
Drawing 10 simulations for pkl 8/10 ...
Wrote sims_1Kset1/8_1466116680.12439.pkl
Drawing 10 simulations for pkl 9/10 ...
Wrote sims_1Kset1/9_1466116680.72445.pkl
Drawing 10 simulations for pkl 10/10 ...
Wrote sims_1Kset1/10_1466116681.33809.pkl


**Variable Explanations**:
"1Kset1" is a name that you can freely choose for your specific set of simulations. 
"truetsr=8.0" means that the synthetic curves will get random true time shifts in a range of about 8.0 days around the time shifts of the model lcs. 

## 5. Running Curve-Shifting Methods with Different Techniques 

Have it include the error; commit with the error, scroll down to this script 6; this line doesn't work, put in error message, explain, report on what i will do with the rest; comment out everything htat needs free-spline; this is a bug in PyCS tutorial that we should report --> paste the scripts in; 

We are now going to run three curve-shifting methods on these data sets. To "define" the microlensing models and initial shifts, we make use of one single set of curves, the observed data. 

Let's set some reasonable guess delays. We can obtain these by eye or through the model fit. 

In [10]:
lcs[1].shifttime(-7.0)
lcs[2].shifttime(-22.0)
lcs[3].shifttime(-65.0)

Our first technique will be the free-knot spline technique. First, we will define the microlensing model, like in Step 3. 

In [11]:
pycs.gen.splml.addtolc(lcs[0], knotstep=150)
pycs.gen.splml.addtolc(lcs[1], knotstep=150)
pycs.gen.splml.addtolc(lcs[2], knotstep=150)
pycs.gen.splml.addtolc(lcs[3], knotstep=150)

Now, we will run the free-knot spline technique using the plain copies we made earlier of the data. 

pycs.sim.run.multirun("copies", lcs, spl, optset="spl", tsrand=10.0, keepopt=True)

Now, we will run the free-knot spline technique using the copies of the synthetic light curve sets we made earlier.

In [ ]:
pycs.sim.run.multirun("1Kset1", lcs, spl, optset="spl", tsrand=10.0, keepopt=True)

**Note**: The two lines of code above do not run. This is bug that should be reported to PyCS through an issue on GitHub. Here is the resulting error message. 

`RuntimeError: Knot spacing min = 7.283257, epsilon = 10.000000`

**Note**: When you run this multirun, as shown above, keep `keepopt=True` as shown above. This will make it easier to read the residuals from the synthetic curves with the residuals from the observed data. You can change the name "optset" to anything, though it should reflect the full method, including the settings of the microlensing. 


Our second technique will be the dispersion technique. First, let's run the code needed to define our variables. 

In [12]:
rawdispersionmethod = lambda lc1, lc2 : pycs.disp.disps.linintnp(lc1, lc2, interpdist = 30.0)
dispersionmethod = lambda lc1, lc2 : pycs.disp.disps.symmetrize(lc1, lc2, rawdispersionmethod)
def disp(lcs):
    return pycs.disp.topopt.opt_full(lcs, rawdispersionmethod, nit=5, verbose=True)

Next, let's factor in our microlensing models (polynomials). 

In [13]:
pycs.gen.polyml.addtolc(lcs[0], nparams=2, autoseasonsgap = 60.0)
pycs.gen.polyml.addtolc(lcs[1], nparams=2, autoseasonsgap = 60.0)
pycs.gen.polyml.addtolc(lcs[2], nparams=2, autoseasonsgap = 60.0)
pycs.gen.polyml.addtolc(lcs[3], nparams=2, autoseasonsgap = 60.0)

I replace an existing mircolensing.
I replace an existing mircolensing.
I replace an existing mircolensing.
I replace an existing mircolensing.


Now we will run it using the dispersion technique on our plain copies data, created earlier. 

In [ ]:
pycs.sim.run.multirun("copies", lcs, disp, optset="disp", tsrand=10.0)

I have found 64 simulation pickles in sims_copies.
I'll write my results into the directory sims_copies_opt_disp.
(The latter already exists.)
Initial conditions : 
[Trial/A]|poly/2222|
[Trial/B](-7.000,0.000,0)|poly/2222|
[Trial/C](-22.000,0.000,0)|poly/2222|
[Trial/D](-65.000,0.000,0)|poly/2222|
--- Casino running on simset copies, optset disp ---
Read sims_copies/12_1466113841.34470.pkl
Working for sims_copies_opt_disp/12_1466113841.34470_runresults.pkl, 10 simulations.
Starting the curve shifting on a single CPU, no multiprocessing...
Starting dispersion optimization of :
[Trial/A](-9.382,0.000,0)|poly/2222|
[Trial/B](-7.856,0.000,0)|poly/2222|
[Trial/D](-69.726,0.000,0)|poly/2222|
[Trial/C](-22.732,0.000,0)|poly/2222|
Initial delays :
AB   +1.53 | AD  -60.34 | AC  -13.35 | BD  -61.87 | BC  -14.88 | DC  +46.99
Iteration 1 done, d2 =    3.664
AB   -5.47 | AD  -68.63 | AC  -22.19 | BD  -63.16 | BC  -16.73 | DC  +46.44
Iteration 2 done, d2 =    3.389
AB   -5.64 | AD  -69.05 | AC  -22.

Now we will run it using the dispersion technique on our synthetic light curve data. 

In [ ]:
pycs.sim.run.multirun("1Kset1", lcs, disp, optset="disp", tsrand=10.0)

Finally, we will use our third technique, the regression difference technique. Let's define our variables. Remember, this line needs pymc to work. 

In [ ]:
def regdiff(lcs):
   return pycs.regdiff.multiopt.opt_ts(lcs, pd=5, scale=200.0, verbose=True)

Now we will run it using the regression difference technique on our plain copies data, created earlier. 

In [ ]:
pycs.sim.run.multirun("copies", lcs, regdiff, optset="regdiff", tsrand=10.0)

Now we will run it using the regression difference technique on our synthetic light curve data. 

In [ ]:
pycs.sim.run.multirun("1Kset1", lcs, regdiff, optset="regdiff", tsrand=10.0)

## 6. Reading the Results from the Observed Light Curves

This script will read the results from the copies of the observed light curves. 

In [ ]:
dataresults = [
        pycs.sim.run.collect("sims_copies_opt_disp", "red", "Dispersion-like technique"),
        pycs.sim.run.collect("sims_copies_opt_regdiff", "green", "Regression difference technique")
]

Now, we can turn this into a simple histogram that will give the instrinic variance. It will be saved to a file called "fig_instrinsicvariance.pdf", for readability. The option `dataout=True` will save the delay point estimate, to be used below. 

In [ ]:
pycs.sim.plot.hists(dataresults, r=5.0, nbins=100, showqs=False,
        filename="fig_intrinsicvariance.pdf", dataout=True)

We read the results obtained on the synthetic curves. 

In [ ]:
simresults = [
        pycs.sim.run.collect("sims_1Kset1_opt_disp", "red", "Dispersion-like technique"),
        pycs.sim.run.collect("sims_1Kset1_opt_regdiff", "green", "Regression difference technique")
]

Now we can perform the error analysis. This will be saved to a file called "fig_measvstrue.pdf". The option dataout=True will save the random and systematic error, to be used below.

In [ ]:
pycs.sim.plot.measvstrue(simresults, errorrange=3.5, r=5.0, nbins = 10, binclip=True, binclipr=20.0,
        plotpoints=False, filename="fig_measvstrue.pdf", dataout=True)

With the same data we can also show the relationship between measurements. This will be written to a file called "fig_covplot.pdf".

In [ ]:
pycs.sim.plot.covplot(simresults, filename="fig_covplot.pdf")

Finally we group the information saved by these steps to get the results in form of a summary plot. Let's define our variables. 

In [ ]:
disp = (pycs.gen.util.readpickle("sims_copies_opt_disp_delays.pkl"),
        pycs.gen.util.readpickle("sims_1Kset1_opt_disp_errorbars.pkl"))

regdiff = (pycs.gen.util.readpickle("sims_copies_opt_regdiff_delays.pkl"),
        pycs.gen.util.readpickle("sims_1Kset1_opt_regdiff_errorbars.pkl"))

# spl = (pycs.gen.util.readpickle("sims_copies_opt_spl_delays.pkl"),
#        pycs.gen.util.readpickle("sims_1Kset1_opt_spl_errorbars.pkl"))

Now we can display our plot! It will be saved to a file called "fig_delays.pdf".

In [ ]:
pycs.sim.plot.newdelayplot([disp, regdiff, spl], rplot=6.0, displaytext=True,
        filename = "fig_delays.pdf", refshifts=[{"colour":"gray", "shifts":(0, -5, -20, -70)}])